<a href="https://colab.research.google.com/github/Chienstartup/ISIC_2024/blob/main/Combined_GBMs_and_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, PredefinedSplit
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.ensemble import VotingClassifier
import catboost as cb

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
sampled_df = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/ISIC_2024/main/meta_data/df_train_filtered.csv')
oof_prediction = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/ISIC_2024/main/meta_data/384_v2b1_5k.csv')

In [6]:
sampled_df = sampled_df.merge(oof_prediction[['isic_id', 'oof_prediction']], on='isic_id')

In [8]:
feature_cols = ['clin_size_long_diam_mm', 'tbp_lv_H_between_std', 'lesion_visibility_score', 'tbp_lv_deltaLBnorm_between_std',
 'age_approx_zscore', 'tbp_lv_B', 'avg_contrast', 'hue_contrast', 'anatom_site_general_encoded', 'attribution_encoded',
 'tbp_lv_C', 'tbp_lv_nevi_confidence_between_std', 'tbp_lv_B_between_std', 'tbp_lv_Aext','oof_prediction']

target_col = 'target'

<h2>Competition: Partial AUC Metric

In [9]:
def custom_metric_binary(y_true, y_pred):
    y_hat = y_pred[:, 1] if y_pred.ndim > 1 else y_pred
    y_true_binary = y_true

    min_tpr = 0.80
    max_fpr = 1 - min_tpr

    v_gt = 1 - y_true_binary
    v_pred = 1 - y_hat

    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

    return partial_auc

<h2> LGB Hypertuning

In [10]:
# def lgb_objective(trial):
#     params = {
#         'objective': 'binary',
#         'num_class': 1,
#         'n_estimators': 200,
#         'verbosity': -1,
#         'boosting_type': 'gbdt',
#         'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
#         'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 4, 8),
#         'num_leaves': trial.suggest_int('num_leaves', 16, 256),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
#         'colsample_bynode': trial.suggest_float('colsample_bynode', 0.4, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 0.8, 4.0),
#         'device': 'cpu'
#     }

#     X = sampled_df[feature_cols]
#     y = sampled_df[target_col]
#     fold = sampled_df['fold']

#     ps = PredefinedSplit(fold)

#     cv_scores = []
#     for train_index, val_index in ps.split():
#         X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#         y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#         model = lgb.LGBMClassifier(**params, random_state=42)
# #         model = lgb.LGBMClassifier(**params)
#         model.fit(X_train, y_train)

#         y_pred = model.predict_proba(X_val)
#         score = custom_metric_binary(y_val, y_pred)
#         cv_scores.append(score)

#     cv_score = np.mean(cv_scores)
#     print('cv_score (Custom metric for target 1): ', cv_score)

#     return cv_score

# # Create an Optuna study and optimize
# study = optuna.create_study(direction='maximize')
# with tqdm(total=100, desc="Optimization Progress") as pbar:
#     study.optimize(lgb_objective, n_trials=100, callbacks=[lambda study, trial: pbar.update(1)])

# # Print the best parameters and best score
# print("Best parameters: ", study.best_params)
# print("Best custom metric score for target 1: ", study.best_value)

Best custom metric score for target 1:  0.1678834053526776

In [11]:
lgb_params = {
        'verbosity':        -1,
        'objective': 'binary',
        'num_class': 1,
        'boosting_type':    'gbdt',
        'n_estimators': 200,
         'lambda_l1': 0.019896528655492955,
        'lambda_l2': 2.2749018975450874,
        'learning_rate': 0.06290066481867992,
        'max_depth': 7,
        'num_leaves': 172,
        'colsample_bytree': 0.5781905414616991,
        'colsample_bynode': 0.46898265874724626,
        'bagging_fraction': 0.8137092082350226,
        'bagging_freq': 7,
        'min_child_samples': 65,
        'scale_pos_weight': 3.028819756924015,
        'device':'cpu'}

lgb_model = Pipeline([
    ('classifier', lgb.LGBMClassifier(**lgb_params, random_state=42)),
])

<h2> CAT Hypertuning

In [12]:
# def cb_objective(trial):
#     params = {
#         'loss_function': 'Logloss',
#         'iterations': 200,
#         'verbose': False,
#         'task_type': 'CPU',
#         'max_depth': trial.suggest_int('max_depth', 4, 8),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
#         'subsample': trial.suggest_float('subsample', 0.4, 1.0),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
#         'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 0.8, 4.0),
#         'bootstrap_type': 'Bernoulli',
#     }

#     X = sampled_df[feature_cols]
#     y = sampled_df[target_col]
#     fold = sampled_df['fold']

#     ps = PredefinedSplit(fold)

#     cv_scores = []
#     for train_index, val_index in ps.split():
#         X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#         y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#         model = CatBoostClassifier(**params, random_seed=42)
#         model.fit(X_train, y_train)

#         y_pred = model.predict_proba(X_val)[:, 1]  #
#         score = custom_metric_binary(y_val, y_pred)
#         cv_scores.append(score)

#     cv_score = np.mean(cv_scores)
#     print('cv_score (Custom metric for binary classification): ', cv_score)

#     return cv_score


# study = optuna.create_study(direction='maximize')
# n_trials = 100

# with tqdm(total=n_trials, desc="Optimization Progress") as pbar:
#     def tqdm_callback(study, trial):
#         pbar.update(1)
#     study.optimize(cb_objective, n_trials=n_trials, callbacks=[tqdm_callback])

# print("Best parameters: ", study.best_params)
# print("Best custom metric score for target 1: ", study.best_value)

Best custom metric score for target 1:  0.16614108059041355

In [13]:
cb_params = {
            'loss_function': 'Logloss',
             'verbose':           False,
            'iterations': 200,
            'max_depth': 5,
            'learning_rate': 0.07466914505150628,
            'l2_leaf_reg': 8.539305470329156,
            'subsample': 0.8984175664471633,
            'min_data_in_leaf': 36,
            'scale_pos_weight': 3.668365584570356,
             'bootstrap_type': 'Bernoulli',
             'task_type': 'CPU',
            }

cb_model = Pipeline([
    ('classifier', cb.CatBoostClassifier(**cb_params, random_seed=42)),
])

<h2> XGB Hypertuning

In [14]:
# def xgb_objective(trial):
#     params = {
#         'objective': 'binary:logistic',
#         'enable_categorical': True,
#         'tree_method': 'hist',
#         'random_state': 42,
#         'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
#         'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
#         'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0),
#         'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 0.8, 4.0),
#         'n_estimators': 200,
#     }

#     X = sampled_df[feature_cols]
#     y = sampled_df[target_col]
#     fold = sampled_df['fold']

#     ps = PredefinedSplit(fold)

#     cv_scores = []
#     for train_index, val_index in ps.split():
#         X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#         y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#         model = xgb.XGBClassifier(**params)
#         model.fit(X_train, y_train)

#         y_pred = model.predict_proba(X_val)[:, 1]
#         score = custom_metric_binary(y_val, y_pred)
#         cv_scores.append(score)

#     cv_score = np.mean(cv_scores)
#     print('cv_score (Custom metric for binary classification): ', cv_score)

#     return cv_score


# study = optuna.create_study(direction='maximize')
# n_trials = 100


# with tqdm(total=n_trials, desc="Optimization Progress") as pbar:
#     def tqdm_callback(study, trial):
#         pbar.update(1)
#     study.optimize(xgb_objective, n_trials=n_trials, callbacks=[tqdm_callback])


# print("Best parameters: ", study.best_params)
# print("Best custom metric score for binary classification: ", study.best_value)

Best custom metric score for binary classification:  0.16813904672815277

In [15]:
xgb_params = {
            'objective': 'binary:logistic',
            'enable_categorical': True,
            'tree_method':        'hist',
            'random_state':       42,
            'learning_rate': 0.05566033566180181,
            'lambda': 4.62951764198257,
            'alpha': 0.19502364654812793,
            'max_depth': 8,
            'subsample': 0.726958618773603,
            'colsample_bytree': 0.884578248658937,
            'colsample_bylevel': 0.7790735100475025,
            'colsample_bynode': 0.7638419540477621,
            'scale_pos_weight': 2.5112778372226665,
             'n_estimators': 200}

xgb_model = Pipeline([
    ('classifier', xgb.XGBClassifier(**xgb_params)),
])

In [16]:
estimator = VotingClassifier([
    ('lgb', lgb_model), ('cb', cb_model), ('xgb', xgb_model),
], voting='soft')

In [17]:
from sklearn.metrics import roc_auc_score, make_scorer

custom_scorer = make_scorer(custom_metric_binary, needs_proba=True, greater_is_better=True)

X = sampled_df[feature_cols]
y = sampled_df[target_col]
fold = sampled_df['fold']
ps = PredefinedSplit(fold)

val_score = cross_val_score(estimator, X, y, cv=ps, scoring=custom_scorer)

print(np.mean(val_score), val_score)

0.18860815147321544 [0.19276057 0.19173353 0.18604825 0.181199   0.1912994 ]
